In [1]:
import sys

sys.path.append("../..")

from plot import *

In [2]:
n = 100000000
estimate_quantile = 0.9935

means = {"U": 5.589, "V": 0.018}
stds = {"U": 9.832, "V": 3.232}


In [3]:
data = pd.read_feather(f"UV-{estimate_quantile}-{n}.ft")
df = data.copy()
data.head()


,time,lev,lat,lon,U_est,V_est,U,V
0,1131,24,156,542,-5.492188,-1.463867,-5.203125,-1.026367
1,132,16,121,383,0.241455,-0.376465,0.551758,-0.787109
2,1814,7,342,319,9.968750,4.695312,9.671875,5.156250
3,309,31,186,45,-8.609375,-0.665527,-8.687500,-0.316895
4,822,30,286,284,0.281006,0.276367,-0.011658,0.172607


In [13]:
df["lat"] = (data["lat"] * 0.5) - 90
df["lat"] /= 90
df["lon"] = (data["lon"] * 0.625) - 180
df["lon"] /= 180
df["lev"] /= 36

df["lev"] = df["lev"].astype("float16")
df["lat"] = df["lat"].astype("float16")
df["lon"] = df["lon"].astype("float16")

df.head()


,time,lev,lat,lon,U_est,V_est,U,V
0,1131,0.018524,-0.133301,0.881836,-5.492188,-1.463867,-5.203125,-1.026367
1,132,0.012344,-0.327881,0.329834,0.241455,-0.376465,0.551758,-0.787109
2,1814,0.005402,0.899902,0.107666,9.968750,4.695312,9.671875,5.156250
3,309,0.023926,0.033325,-0.843750,-8.609375,-0.665527,-8.687500,-0.316895
4,822,0.023148,0.588867,-0.013885,0.281006,0.276367,-0.011658,0.172607


In [15]:
df["U"] -= means["U"]
df["U"] /= stds["U"]

df["V"] -= means["V"]
df["V"] /= stds["V"]

df["U_est"] -= means["U"]
df["U_est"] /= stds["U"]

df["V_est"] -= means["V"]
df["V_est"] /= stds["V"]

df.head()


,time,lev,lat,lon,U_est,V_est,U,V
0,1131,0.018524,-0.133301,0.881836,-1.126953,-0.458252,-1.098633,-0.322998
1,132,0.012344,-0.327881,0.329834,-0.543945,-0.122070,-0.512695,-0.249146
2,1814,0.005402,0.899902,0.107666,0.445557,1.446289,0.415283,1.588867
3,309,0.023926,0.033325,-0.843750,-1.445312,-0.211426,-1.453125,-0.103638
4,822,0.023148,0.588867,-0.013885,-0.540039,0.079895,-0.569824,0.047821


In [22]:
df["day %"] = (data["time"] % 8) / 8
df["year %"] = data["time"] / 2920

df["day %"] *= np.pi * 2
df["year %"] *= np.pi * 2

df["sin_day"] = np.sin(df["day %"]).astype("float16")
df["cos_day"] = np.cos(df["day %"]).astype("float16")
df["sin_year"] = np.sin(df["year %"]).astype("float16")
df["cos_year"] = np.cos(df["year %"]).astype("float16")

del df["time"]
del df["day %"]
del df["year %"]

df.head()


,lev,lat,lon,U_est,V_est,U,V,sin_day,cos_day,sin_year,cos_year
0,0.018524,-0.133301,0.881836,-1.126953,-0.458252,-1.098633,-0.322998,0.707031,-0.707031,0.650391,-0.759766
1,0.012344,-0.327881,0.329834,-0.543945,-0.122070,-0.512695,-0.249146,0.000000,-1.000000,0.280273,0.959961
2,0.005402,0.899902,0.107666,0.445557,1.446289,0.415283,1.588867,-1.000000,-0.000000,-0.689941,-0.723633
3,0.023926,0.033325,-0.843750,-1.445312,-0.211426,-1.453125,-0.103638,-0.707031,-0.707031,0.617188,0.787109
4,0.023148,0.588867,-0.013885,-0.540039,0.079895,-0.569824,0.047821,-1.000000,-0.000000,0.980469,-0.196655


In [30]:
frland = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FRLAND",
                                         time=0, level=0, folder="..").astype("float16")
frocean = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FROCEAN",
                                          time=0, level=0, folder="..").astype("float16")
frlake = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FRLAKE",
                                         time=0, level=0, folder="..").astype("float16")
frlandice = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FRLANDICE",
                                            time=0, level=0, folder="..").astype("float16")
phis = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="PHIS",
                                       time=0, level=0, folder="..")
sgh = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="SGH",
                                      time=0, level=0, folder="..")

phis -= phis.mean()
phis /= phis.std()
phis = phis.astype("float16")

sgh -= sgh.mean()
sgh /= sgh.std()
sgh = sgh.astype("float16")

indices = data[["lat", "lon"]].values.T

df["frland"] = frland[*indices]
df["frocean"] = frocean[*indices]
df["frlake"] = frlake[*indices]
df["frlandice"] = frlandice[*indices]
df["phis"] = phis[*indices]
df["sgh"] = sgh[*indices]

df.head()


,lev,lat,lon,U_est,V_est,U,V,sin_day,cos_day,sin_year,cos_year,frland,frocean,frlake,frlandice,phis,sgh
0,0.018524,-0.133301,0.881836,-1.126953,-0.458252,-1.098633,-0.322998,0.707031,-0.707031,0.650391,-0.759766,0.000000,1.000000,0.000000,0.000000,-0.447754,-0.393799
1,0.012344,-0.327881,0.329834,-0.543945,-0.122070,-0.512695,-0.249146,0.000000,-1.000000,0.280273,0.959961,0.000000,1.000000,0.000000,0.000000,-0.447754,-0.393799
2,0.005402,0.899902,0.107666,0.445557,1.446289,0.415283,1.588867,-1.000000,-0.000000,-0.689941,-0.723633,0.005371,0.987793,0.000366,0.006531,-0.446533,-0.332031
3,0.023926,0.033325,-0.843750,-1.445312,-0.211426,-1.453125,-0.103638,-0.707031,-0.707031,0.617188,0.787109,0.000000,1.000000,0.000000,0.000000,-0.447754,-0.393799
4,0.023148,0.588867,-0.013885,-0.540039,0.079895,-0.569824,0.047821,-1.000000,-0.000000,0.980469,-0.196655,0.980469,0.018158,0.001513,0.000000,-0.291992,0.514160


In [31]:
df.dtypes

lev          float16
lat          float16
lon          float16
U_est        float16
V_est        float16
U            float16
V            float16
sin_day      float16
cos_day      float16
sin_year     float16
cos_year     float16
frland       float16
frocean      float16
frlake       float16
frlandice    float16
phis         float16
sgh          float16
dtype: object

In [32]:
# N - normalized
# G - geographical features
# CT - cyclic time

df.to_feather(f"UV-NGCT-{estimate_quantile}-{n}.ft")

In [ ]:
df.astype("float32").describe()
